In [1]:
import numpy as np
from simplifiedLikelihoods import Data,UpperLimitComputer
import pandas as pd

In [2]:
recastFile = '../scanResults_axial.pcl'

### Define MET bins and their order

In [3]:
metBins = [250,  280,  310,  340,  370,  400,  430,  470,  510, 550,  590,  640,  690,  
            740,  790,  840,  900,  960, 1020, 1090, 1160, 1250, 1400]

### Load CMS data

In [4]:
cmsYieldFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:Yields(Monojet).csv'
bgYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=5,
                          skip_footer=67,names=True,dtype=None,encoding=None)

dataYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=73,
                           names=True,dtype=None,encoding=None)

covarianceFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:covariancematrix(Monojet).csv'
cov = np.genfromtxt(covarianceFile,delimiter=',',skip_header=5,
                           names=True,dtype=None,encoding='UTF-8')

### Extract covariance matrix

In [5]:
srOrder = cov['Second_Bin'][:66]
covMatrix = np.zeros((len(srOrder),len(srOrder)))
for pt in cov:
    i = np.where(srOrder == pt['First_Bin'])
    j = np.where(srOrder == pt['Second_Bin'])
    covMatrix[i,j] = pt['Covariance']

### Get number of observed and expected (BG) events

In [6]:
nobs = [int(dataYields[dataYields['Bin'] == sr]['Data_yield'][0]) for sr in srOrder]
nbg = [float(bgYields[bgYields['Bin'] == sr]['Background_yield'][0]) for sr in srOrder]

### Load Recast Data

In [7]:
recastData = pd.read_pickle(recastFile)

### Get all model points

In [8]:
models = []
mCols = ['Coupling','Mode','$m_{med}$','$m_{DM}$']
for row in recastData[mCols].values:
    m = dict(zip(mCols,row.tolist()))
    if m not in models:
        models.append(m)
# print(len(models))

### Loop over model points and compute UL on mu

In [9]:
ulComp = UpperLimitComputer()

for m in models:
    dfModel = recastData.loc[(recastData[list(m)] == pd.Series(m)).all(axis=1)]
    ns = []
    for sr in srOrder:
        dataset,ibin = sr.split('_')[1:]
        dataset = eval(dataset)
        ibin = eval(ibin.replace('bin',''))
        binLabel = 'bin_%1.1f_%1.1f' %(metBins[ibin],metBins[ibin+1])
        signalYield = dfModel[dfModel['Data-takingperiod'] == dataset][binLabel].iloc[0]
        ns.append(signalYield)
    ns = np.array(ns)    
    data = Data(observed=nobs, backgrounds=nbg, covariance=covMatrix, 
                nsignal=ns,deltas_rel=0.0)
    dataExp = Data(observed=[int(b) for b in nbg], backgrounds=nbg, covariance=covMatrix, 
                   nsignal=ns,deltas_rel=0.0)    
    try:
        ul = ulComp.getUpperLimitOnMu(data)
    except:
        print('Error computing ul for model:\n',m,'\n')
        ul = None
    try:
        ulExp = ulComp.getUpperLimitOnMu(dataExp)    
    except:
        print('Error computing ulExp for model:\n',m,'\n')
        ulExp = None
        
    recastData.loc[dfModel.index,'$\mu^{UL}_{obs}$'] = ul
    recastData.loc[dfModel.index,'$\mu^{UL}_{exp}$'] = ulExp

In [10]:
recastData

,Coupling,Mode,$m_{med}$,$m_{DM}$,Data-takingperiod,Luminosity (1/fb),bin_250.0_280.0,bin_250.0_280.0_ErrorPlus,bin_250.0_280.0_ErrorMinus,bin_280.0_310.0,...,Tauveto,Bjetveto,Photonveto,"$\Delta \phi (jet,p_{T}^{miss})>0.5$ rad",LeadingAK4jet$p_{T}>100$GeV,LeadingAK4jet$\eta<2.4$,HCALmitigation(jets),HCALmitigation($\phi^{miss}$),$\mu^{UL}_{obs}$,$\mu^{UL}_{exp}$
0,Axial,DM+QCDjets,100.0,1.0,2018,59.7,251104.487828,21601.521276,21601.521276,142777.647570,...,0.119180,0.104340,0.103476,0.102896,0.100214,0.100214,0.097009,0.083431,0.017710,0.008568
1,Axial,DM+QCDjets,100.0,1.0,2017,41.5,271722.684848,23863.891338,23863.891338,116392.591297,...,0.129459,0.116259,0.115025,0.114846,0.112331,0.112331,0.112331,0.112331,0.017710,0.008568
2,Axial,DM+QCDjets,100.0,1.0,2016,36.0,168058.900804,17606.452597,17606.452597,147370.357757,...,0.122386,0.108214,0.107884,0.107074,0.105500,0.105500,0.105500,0.105500,0.017710,0.008568
3,Axial,DM+QCDjets,100.0,30.0,2018,59.7,191669.394832,16395.086037,16395.086037,90514.228525,...,0.118952,0.108331,0.107125,0.106105,0.104741,0.104741,0.101216,0.083459,0.021851,0.011197
4,Axial,DM+QCDjets,100.0,30.0,2017,41.5,152244.944167,13753.523201,13753.523201,106533.419009,...,0.125361,0.112053,0.111839,0.111114,0.109761,0.109761,0.109761,0.109761,0.021851,0.011197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Axial,DM+QCDjets,2500.0,1000.0,2017,41.5,15.306873,1.274373,1.274373,10.348824,...,0.377676,0.334588,0.324343,0.322794,0.320203,0.320203,0.320203,0.320203,13.536619,7.210914
200,Axial,DM+QCDjets,2500.0,1000.0,2016,36.0,12.548104,1.120931,1.120931,9.717659,...,0.382897,0.334718,0.322672,0.323224,0.322659,0.322659,0.322659,0.322659,13.536619,7.210914
201,Axial,DM+QCDjets,2500.0,1500.0,2018,59.7,0.028364,0.004259,0.004259,0.023452,...,0.368547,0.322625,0.311357,0.310497,0.308954,0.308954,0.301113,0.265798,7646.472673,4771.557494
202,Axial,DM+QCDjets,2500.0,1500.0,2017,41.5,0.026579,0.003746,0.003746,0.017602,...,0.360645,0.313921,0.302280,0.302946,0.299669,0.299669,0.299669,0.299669,7646.472673,4771.557494


In [11]:
recastData.to_pickle(recastFile)